In [ ]:
import pandas as pd

# Load the dataset using ISO-8859-1 encoding
df = pd.read_csv("data.csv", encoding='ISO-8859-1')

# Display basic info
print(df.info())
print(df.head())



# Drop rows where critical fields are missing
df_cleaned = df.dropna(subset=['state', 'location', 'date'])

# Fill missing numeric values with the column mean
num_cols = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']
df_cleaned[num_cols] = df_cleaned[num_cols].fillna(df_cleaned[num_cols].mean())

# Fill missing categorical values with the mode
cat_cols = ['agency', 'type', 'location_monitoring_station']
for col in cat_cols:
    df_cleaned[col] = df_cleaned[col].fillna(df_cleaned[col].mode()[0])

# Standardize column names
df_cleaned.columns = [col.lower().strip().replace(" ", "_") for col in df_cleaned.columns]




# Simulate a dataset of state codes
state_codes = pd.DataFrame({
    'state': df_cleaned['state'].unique(),
    'state_code': range(1, len(df_cleaned['state'].unique()) + 1)
})

# Merge on 'state'
df_merged = pd.merge(df_cleaned, state_codes, on='state', how='left')






# Convert 'date' to datetime
df_merged['date'] = pd.to_datetime(df_merged['date'], errors='coerce')

# Extract year and month
df_merged['year'] = df_merged['date'].dt.year
df_merged['month'] = df_merged['date'].dt.month

# Create a simplified pollution index
df_merged['pollution_index'] = df_merged[['so2', 'no2', 'rspm']].mean(axis=1)




# Convert negative values to positive in pollution columns
for col in ['so2', 'no2', 'rspm', 'spm', 'pm2_5']:
    df_merged[col] = df_merged[col].apply(lambda x: abs(x) if pd.notnull(x) else x)

# Capitalize the pollution type field for consistency
df_merged['type'] = df_merged['type'].str.title()




# View the first 10 rows of the final dataset
print(df_merged.head(10))

# Check summary statistics
print(df_merged.describe(include='all'))

# Check if there are still any missing values
print(df_merged.isnull().sum())
